In [24]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import statsmodels.api as sm
import statsmodels.formula.api as smf

import seaborn as sns
sns.set_context('poster')
%matplotlib inline
sns.set_style('ticks')

/Users/iwang/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [3]:
all_emails = pd.read_csv('./response_data/emails_with_times.csv')
#response_data = pd.read_csv('./regress3.csv')

In [4]:
all_emails.tail()

,Unnamed: 0,from,sector,sent,received,email,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,time_dff,hours
1316,1316,Tyrone James,autocare,2017-11-29 22:15:00,2017-11-30 06:00:00,1 million dollars. >,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 07:45:00.000000000,7.0
1317,1317,Tyrone James,autocare,2017-11-29 10:09:00,2017-11-30 09:48:00,We do not do a 4- wheel alignment. Thank You....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 23:39:00.000000000,23.0
1318,1318,Tyrone James,tutoring,2017-11-29 21:47:00,2017-11-30 04:23:00,Sorry I guess you have the wrong address Www....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 06:36:00.000000000,6.0
1319,1319,Tyrone James,babysitting,2017-11-29 09:41:00,2017-12-01 23:54:00,"Hi Tyrone,¬†That would be considered drop-in w...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 days 14:13:00.000000000,62.0
1320,1320,Tyrone James,plumbing,2017-11-30 09:27:00,2017-12-01 08:35:00,"Hello Tyrone, Thank you for your email. We ar...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 23:08:00.000000000,23.0


In [5]:
# Drop all the irrelvent columns
all_emails = all_emails.loc[:, ['from','sector','sent','received','email']]

In [6]:
all_emails.head()

,from,sector,sent,received,email
0,Gary Chow,plumbing,2017-11-30 11:26:00,2017-12-01 05:57:00,"Gary, Are you in Santa Clara, CA? Our plumbin..."
1,Gary Chow,tutoring,2017-11-29 21:56:00,2017-11-29 23:40:00,"Hi Gary, Would this be for Algebra 1? Is your ..."
2,Gary Chow,tutoring,2017-11-29 21:59:00,2017-11-29 21:59:00,"<div dir=""ltr"">Hello,¬†<div><br></div><div>Dav..."
3,Gary Chow,tutoring,2017-11-29 21:52:00,2017-11-30 09:51:00,Hello!! Kumon is not a tutoring service ! We a...
4,Gary Chow,autocare,2017-11-29 10:13:00,2017-11-30 08:08:00,"Hi Gary, We sublet to a neighbor garage, but ..."


In [7]:
all_emails['received'] = pd.to_datetime(all_emails['received'])
all_emails['sent'] = pd.to_datetime(all_emails['sent'])

In [8]:
all_emails.isnull().any()

from        False
sector      False
sent        False
received     True
email        True
dtype: bool

In [9]:
all_emails.loc[all_emails['email'].isnull(),:]

,from,sector,sent,received,email
529,LaToya Johnson,plumbing,2017-11-30 09:26:00,2017-12-01 10:10:00,NaN


In [10]:
all_emails.loc[all_emails['received'].isnull(),:]

,from,sector,sent,received,email
187,James Hoyt,gardening,2017-11-29 21:47:00,NaT,Hello! Thank you for your inquiry to Simplif...


In [11]:
all_emails.shape

(1321, 5)

In [12]:
# Only 2 values are nan. Comfortable to remove the rows
all_emails.dropna(inplace=True)

In [13]:
print all_emails.shape
print all_emails.isnull().any()

(1319, 5)
from        False
sector      False
sent        False
received    False
email       False
dtype: bool


In [14]:
all_emails.describe()

,from,sector,sent,received,email
count,1319,1319,1319,1319,1319
unique,9,8,206,1002,1308
top,Michelle Chen,autocare,2017-11-30 21:43:00,1970-01-01 00:00:00,0
freq,188,408,19,6,6
first,NaN,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,NaN
last,NaN,NaN,2017-11-30 21:59:00,2017-12-11 18:35:00,NaN


In [15]:
# Remove records with no names
all_emails = all_emails.loc[all_emails['from']!='0',:]

In [16]:
all_emails.sector.unique()

array(['plumbing', 'tutoring', 'autocare', 'babysitting', 'gardening',
       'cleaning', 'Estimate '], dtype=object)

In [17]:
# Remove one record where the sector is estimate
all_emails = all_emails.loc[all_emails['sector']!='Estimate ',:]

In [18]:
all_emails["time_dff"] = all_emails['received']- all_emails['sent']

In [19]:
all_emails['hours_diff'] = all_emails['time_dff'].astype('timedelta64[h]')

In [20]:
all_emails.head()

,from,sector,sent,received,email,time_dff,hours_diff
0,Gary Chow,plumbing,2017-11-30 11:26:00,2017-12-01 05:57:00,"Gary, Are you in Santa Clara, CA? Our plumbin...",18:31:00,18.0
1,Gary Chow,tutoring,2017-11-29 21:56:00,2017-11-29 23:40:00,"Hi Gary, Would this be for Algebra 1? Is your ...",01:44:00,1.0
2,Gary Chow,tutoring,2017-11-29 21:59:00,2017-11-29 21:59:00,"<div dir=""ltr"">Hello,¬†<div><br></div><div>Dav...",00:00:00,0.0
3,Gary Chow,tutoring,2017-11-29 21:52:00,2017-11-30 09:51:00,Hello!! Kumon is not a tutoring service ! We a...,11:59:00,11.0
4,Gary Chow,autocare,2017-11-29 10:13:00,2017-11-30 08:08:00,"Hi Gary, We sublet to a neighbor garage, but ...",21:55:00,21.0


In [21]:
all_emails['from'].unique()

array(['Gary Chow', 'James Hoyt', 'Juan Sanchez', 'LaToya Johnson',
       'Maria Rodriguez', 'Michelle Chen', 'Sarah Anderson', 'Tyrone James'], dtype=object)

In [22]:
# Add gender identifiers
male_names = ['Gary Chow', 'James Hoyt', 'Juan Sanchez', 'Tyrone James']
all_emails['genders'] = [0 if name in male_names else 1 for name in all_emails['from']]

# Add ethnicity
asian_names = ['Gary Chow', 'Michelle Chen']
all_emails['asian'] = [1 if name in asian_names else 0 for name in all_emails['from']]
white_names = ['James Hoyt', 'Sarah Anderson']
all_emails['white'] = [1 if name in white_names else 0 for name in all_emails['from']]
latino_names = ['Juan Sanchez', 'Maria Rodriguez']
all_emails['latino'] = [1 if name in latino_names else 0 for name in all_emails['from']]
black_names = ['Tyrone James', 'LaToya Johnson']
all_emails['black'] = [1 if name in black_names else 0 for name in all_emails['from']]


In [23]:
all_emails.describe()

,time_dff,hours_diff,genders,asian,white,latino,black
count,1312,1312.000000,1312.000000,1312.000000,1312.000000,1312.000000,1312.000000
mean,1 days 02:22:51.402439,25.919207,0.521341,0.262957,0.279726,0.233232,0.224085
std,1 days 09:39:42.192464,33.632707,0.499735,0.440407,0.449035,0.423050,0.417138
min,0 days 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0 days 10:47:45,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0 days 19:44:30,19.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1 days 02:41:15,26.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,12 days 08:46:00,296.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### EDA on sectors

In [26]:
trace = go.Bar(x=all_emails.sector.unique(), y=all_emails.sector.value_counts())
layout = dict(title='Sector Breakdown on E-mails Received', xaxis=dict(title='Sector'), yaxis=dict(title='Count'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig)

We received similar amount of responses from tutoring, autocare, babysitting, and gardening. We received fewest responses on cleaning, possibly because cleaning is more of a 'word of mouth' business and less established. We received many more responses from plumbing. 

#### EDA on Response Time

In [24]:
trace = go.Histogram(x=all_emails.hours_diff, xbins=dict(start=np.min(all_emails.hours_diff), size=1, end=np.max(all_emails.hours_diff)), histnorm='probability')
layout = dict(title='Emails Response Time Distribution', xaxis=dict(title='Number of Hours'), yaxis=dict(title='Proportion Count'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig)

The distribution is consistent with our expectation where most businesses responded within a day. Also, few businesses responded more than a week later, which caused the long right tail. 

In addition, we noticed three spikes in response time.
1. Responded immediately at night
2. The next morning around 8-11am 
3. About 22 hours after sent time, which was around 8pm the next day

#### Response time by gender

In [25]:
# Response time based on gender
gender_emails = all_emails.groupby(by='genders').mean()

In [26]:
print 'Average response time for males: %0.2f' % gender_emails.iloc[0,0]
print 'Average response time for females: %0.2f' % gender_emails.iloc[1,0]

Average response time for males: 27.29
Average response time for females: 24.66


In [27]:
trace1 = go.Histogram(x=all_emails.loc[all_emails['genders']==0, 'hours_diff'],xbins=dict(start=0, size=2, end=250), opacity=0.75, histnorm='probability', name='male')
trace2 = go.Histogram(x=all_emails.loc[all_emails['genders']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=250), opacity=0.75, histnorm='probability', name='female')
layout = dict(title='Response Time Distribution by Gender', barmode='overlay', xaxis=dict(title='Number of Hours'), yaxis=dict(title='Proportion Count'))
fig = dict(data=[trace1, trace2], layout=layout)
plotly.offline.iplot(fig)

The distribution shape was similar between male and female, with males exhibited more right skewed time responses

#### Response time by ethnicity

In [28]:
# Response time based on ethnicity
print 'Average response time for Caucasians: %0.2f' % all_emails.groupby(by='white').mean().iloc[1,0]
print 'Average response time for Asians: %0.2f' % all_emails.groupby(by='asian').mean().iloc[1,0]
print 'Average response time for African-American: %0.2f' % all_emails.groupby(by='black').mean().iloc[1,0]
print 'Average response time for Latinos: %0.2f' % all_emails.groupby(by='latino').mean().iloc[1,0]

Average response time for Caucasians: 27.68
Average response time for Asians: 22.85
Average response time for African-American: 26.48
Average response time for Latinos: 26.73


In [29]:
trace1 = go.Histogram(x=all_emails.loc[all_emails['white']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), marker=dict(color='rgb(58,200,225)'), opacity=0.75, histnorm='probability', name='white')
trace2 = go.Histogram(x=all_emails.loc[all_emails['black']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='black')
layout = dict(title='Response Time Distribution by Gender', barmode='overlay', xaxis=dict(title='Number of Hours'), yaxis=dict(title='Proportion Count'))
fig = plotly.tools.make_subplots(rows=2, cols=2)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

trace1 = go.Histogram(x=all_emails.loc[all_emails['white']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), marker=dict(color='rgb(58,200,225)'), opacity=0.75, histnorm='probability', name='white')
trace2 = go.Histogram(x=all_emails.loc[all_emails['latino']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='latino')
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 2)

trace1 = go.Histogram(x=all_emails.loc[all_emails['white']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), marker=dict(color='rgb(58,200,225)'), opacity=0.6, histnorm='probability', name='white')
trace2 = go.Histogram(x=all_emails.loc[all_emails['asian']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.6, histnorm='probability', name='asian')
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(dict(title='Response Time Distribution by Ethnicity', barmode='overlay'))
plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [30]:
all_emails.head()

,from,sector,sent,received,email,time_dff,hours_diff,genders,asian,white,latino,black
0,Gary Chow,plumbing,2017-11-30 11:26:00,2017-12-01 05:57:00,"Gary, Are you in Santa Clara, CA? Our plumbin...",18:31:00,18.0,0,1,0,0,0
1,Gary Chow,tutoring,2017-11-29 21:56:00,2017-11-29 23:40:00,"Hi Gary, Would this be for Algebra 1? Is your ...",01:44:00,1.0,0,1,0,0,0
2,Gary Chow,tutoring,2017-11-29 21:59:00,2017-11-29 21:59:00,"<div dir=""ltr"">Hello,¬†<div><br></div><div>Dav...",00:00:00,0.0,0,1,0,0,0
3,Gary Chow,tutoring,2017-11-29 21:52:00,2017-11-30 09:51:00,Hello!! Kumon is not a tutoring service ! We a...,11:59:00,11.0,0,1,0,0,0
4,Gary Chow,autocare,2017-11-29 10:13:00,2017-11-30 08:08:00,"Hi Gary, We sublet to a neighbor garage, but ...",21:55:00,21.0,0,1,0,0,0


In [31]:
# Response time based on gender and ethnicity
print 'Average response time for Caucasian Males: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==0) & (all_emails['white']==1), 'hours_diff'])
print 'Average response time for Caucasian Females: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==1) & (all_emails['white']==1), 'hours_diff'])
print 'Average response time for Asian Males: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==0) & (all_emails['asian']==1), 'hours_diff'])
print 'Average response time for Asian Females: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==1) & (all_emails['asian']==1), 'hours_diff'])
print 'Average response time for Latino Males: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==0) & (all_emails['latino']==1), 'hours_diff'])
print 'Average response time for Latino Females: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==1) & (all_emails['latino']==1), 'hours_diff'])
print 'Average response time for Afriacn-American Males: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==0) & (all_emails['black']==1), 'hours_diff'])
print 'Average response time for Afriacn-American Females: %0.2f' % np.mean(all_emails.loc[(all_emails['genders']==1) & (all_emails['black']==1), 'hours_diff'])

Average response time for Caucasian Males: 27.58
Average response time for Caucasian Females: 27.79
Average response time for Asian Males: 27.42
Average response time for Asian Females: 18.99
Average response time for Latino Males: 27.38
Average response time for Latino Females: 26.20
Average response time for Afriacn-American Males: 26.74
Average response time for Afriacn-American Females: 26.19


In [32]:
fig = plotly.tools.make_subplots(rows=4, cols=2)
trace1 = go.Histogram(x=all_emails.loc[(all_emails['white']==1) & (all_emails['genders']==0), 'hours_diff'],xbins=dict(start=0, size=2, end=100), histnorm='probability', name='white male')
trace2 = go.Histogram(x=all_emails.loc[(all_emails['white']==1) & (all_emails['genders']==1), 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='white female')

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

trace1 = go.Histogram(x=all_emails.loc[(all_emails['asian']==1) & (all_emails['genders']==0), 'hours_diff'],xbins=dict(start=0, size=2, end=100), histnorm='probability', name='asian male')
trace2 = go.Histogram(x=all_emails.loc[(all_emails['asian']==1) & (all_emails['genders']==1), 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='asian female')
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace2, 2, 2)

trace1 = go.Histogram(x=all_emails.loc[(all_emails['latino']==1) & (all_emails['genders']==0), 'hours_diff'],xbins=dict(start=0, size=2, end=100), histnorm='probability', name='latino male')
trace2 = go.Histogram(x=all_emails.loc[(all_emails['latino']==1) & (all_emails['genders']==1), 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='latino female')
fig.append_trace(trace1, 3, 1)
fig.append_trace(trace2, 3, 2)

trace1 = go.Histogram(x=all_emails.loc[(all_emails['black']==1) & (all_emails['genders']==0), 'hours_diff'],xbins=dict(start=0, size=2, end=100), histnorm='probability', name='black male')
trace2 = go.Histogram(x=all_emails.loc[(all_emails['black']==1) & (all_emails['genders']==1), 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.75, histnorm='probability', name='black female')
fig.append_trace(trace1, 4, 1)
fig.append_trace(trace2, 4, 2)

fig['layout'].update(dict(title='Response Time Distribution by Ethnicity', barmode='overlay'))
fig['layout']['xaxis1'].update(range=[10, 80])
fig['layout']['xaxis2'].update(range=[10, 80])
fig['layout']['xaxis3'].update(range=[10, 80])
fig['layout']['xaxis4'].update(range=[10, 80])
fig['layout']['xaxis5'].update(range=[10, 80])
fig['layout']['xaxis6'].update(range=[10, 80])
fig['layout']['xaxis7'].update(range=[10, 80])
fig['layout']['xaxis8'].update(range=[10, 80])

plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]
[ (4,1) x7,y7 ]  [ (4,2) x8,y8 ]



#### Regression on gender

In [33]:
all_emails.head()

,from,sector,sent,received,email,time_dff,hours_diff,genders,asian,white,latino,black
0,Gary Chow,plumbing,2017-11-30 11:26:00,2017-12-01 05:57:00,"Gary, Are you in Santa Clara, CA? Our plumbin...",18:31:00,18.0,0,1,0,0,0
1,Gary Chow,tutoring,2017-11-29 21:56:00,2017-11-29 23:40:00,"Hi Gary, Would this be for Algebra 1? Is your ...",01:44:00,1.0,0,1,0,0,0
2,Gary Chow,tutoring,2017-11-29 21:59:00,2017-11-29 21:59:00,"<div dir=""ltr"">Hello,¬†<div><br></div><div>Dav...",00:00:00,0.0,0,1,0,0,0
3,Gary Chow,tutoring,2017-11-29 21:52:00,2017-11-30 09:51:00,Hello!! Kumon is not a tutoring service ! We a...,11:59:00,11.0,0,1,0,0,0
4,Gary Chow,autocare,2017-11-29 10:13:00,2017-11-30 08:08:00,"Hi Gary, We sublet to a neighbor garage, but ...",21:55:00,21.0,0,1,0,0,0


In [34]:
results = smf.ols('hours_diff ~ genders', data=all_emails).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             hours_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.974
Date:                Wed, 20 Dec 2017   Prob (F-statistic):              0.160
Time:                        02:36:37   Log-Likelihood:                -6472.5
No. Observations:                1312   AIC:                         1.295e+04
Df Residuals:                    1310   BIC:                         1.296e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     27.2866      1.408     19.380      0.0

#### Regression on gender with blocking

In [35]:
results = smf.ols('hours_diff ~ genders + sector', data=all_emails).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             hours_diff   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.654
Date:                Wed, 20 Dec 2017   Prob (F-statistic):              0.129
Time:                        02:36:37   Log-Likelihood:                -6468.5
No. Observations:                1312   AIC:                         1.295e+04
Df Residuals:                    1305   BIC:                         1.299e+04
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                23.99

#### Regression on ethnicity

In [36]:
results = smf.ols('hours_diff ~ asian + latino + black', data=all_emails).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             hours_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.495
Date:                Wed, 20 Dec 2017   Prob (F-statistic):              0.214
Time:                        02:36:39   Log-Likelihood:                -6471.4
No. Observations:                1312   AIC:                         1.295e+04
Df Residuals:                    1308   BIC:                         1.297e+04
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     27.6839      1.853     14.942      0.0

#### Regression on ethnicity with blocking

In [37]:
results = smf.ols('hours_diff ~ asian + latino + black + sector', data=all_emails).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             hours_diff   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.264
Date:                Wed, 20 Dec 2017   Prob (F-statistic):              0.258
Time:                        02:36:40   Log-Likelihood:                -6467.5
No. Observations:                1312   AIC:                         1.295e+04
Df Residuals:                    1303   BIC:                         1.300e+04
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                24.30

#### Regression on everything

In [38]:
results = smf.ols('hours_diff ~ genders + asian + latino + black + sector', data=all_emails).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             hours_diff   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.351
Date:                Wed, 20 Dec 2017   Prob (F-statistic):              0.206
Time:                        02:36:43   Log-Likelihood:                -6466.6
No. Observations:                1312   AIC:                         1.295e+04
Df Residuals:                    1302   BIC:                         1.301e+04
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                25.54

In [71]:
gender = response_data.groupby(by='gender').mean()
ethnicity = response_data.groupby(by='a_white').mean()
names = response_data.groupby(by='from').mean()

In [76]:
response_data['from'] = response_data['from'].str.replace('_', ' ')
names = response_data.groupby(by='from').mean()

In [70]:
trace1 = go.Bar(x=['Male','Female'], y=gender['got_response'])
layout = dict(title='Response Rate Comparison by Gender', barmode='overlay', xaxis=dict(title='Gender'), yaxis=dict(title='Response Rate'))
fig = plotly.tools.make_subplots(rows=1, cols=2, subplot_titles=('Response Rate Comparison by Gender', 'Response Rate Comparison by Ethnicity'))
fig.append_trace(trace1, 1, 1)

trace1 = go.Bar(x=['Non-Caucasian','Caucasian'], y=ethnicity['got_response'])
fig.append_trace(trace1, 1, 2)
# fig.append_trace(trace2, 1, 2)

# trace1 = go.Histogram(x=all_emails.loc[all_emails['white']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), marker=dict(color='rgb(58,200,225)'), opacity=0.6, histnorm='probability', name='white')
# trace2 = go.Histogram(x=all_emails.loc[all_emails['asian']==1, 'hours_diff'],xbins=dict(start=0, size=2, end=100), opacity=0.6, histnorm='probability', name='asian')
# fig.append_trace(trace1, 2, 1)
# fig.append_trace(trace2, 2, 1)

# fig['layout'].update(dict(title='Response Time Distribution by Ethnicity', barmode='overlay'))
fig['layout'].update(showlegend=False)
fig['layout']['yaxis2'].update(range=[0, 1])
fig['layout']['yaxis1'].update(title='Response Rate', range=[0, 1])
plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [85]:
trace1 = go.Bar(x=names.index.values, y=names['got_response'])
layout = dict(title='Response Rate Comparison of Individual Names', barmode='overlay', xaxis=dict(title='Names'), yaxis=dict(title='Response Rate', range=[0,1]))
fig = dict(data=[trace1], layout=layout)
plotly.offline.iplot(fig)

In [86]:
response_data.head()

,from,got_response,ethnicity,gender,a_white,asian,hispanic,aa,a_male,female,sector,auto,plumbing,tutoring,babysitting,cleaning,gardening,sent,received,timediff
0,Juan Sanchez,1,hispanic,a_male,0,0,1,0,1,0,babysitting,0,0,0,1,0,0,11/29/2017 9:49,12/11/2017 18:35,296.766667
1,LaToya Johnson,1,afamerican,female,0,0,0,1,0,1,tutoring,0,0,1,0,0,0,11/29/2017 10:01,12/11/2017 18:35,296.566667
2,James Hoyt,1,a_white,a_male,1,0,0,0,1,0,gardening,0,0,0,0,0,1,11/29/2017 9:38,12/11/2017 8:33,286.916667
3,Tyrone James,1,afamerican,a_male,0,0,0,1,1,0,gardening,0,0,0,0,0,1,11/30/2017 9:39,12/11/2017 16:15,270.600000
4,Michelle Chen,1,asian,female,0,1,0,0,0,1,autocare,1,0,0,0,0,0,11/29/2017 22:21,12/11/2017 2:19,267.966667


In [89]:
results = smf.ols('got_response ~ gender + sector', data=response_data).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           got_response   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     63.79
Date:                Wed, 20 Dec 2017   Prob (F-statistic):           1.50e-74
Time:                        03:31:45   Log-Likelihood:                -1909.0
No. Observations:                2858   AIC:                             3832.
Df Residuals:                    2851   BIC:                             3874.
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                 0.73

In [90]:
results = smf.ols('got_response ~ asian + hispanic + aa + sector', data=response_data).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           got_response   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     51.32
Date:                Wed, 20 Dec 2017   Prob (F-statistic):           5.12e-78
Time:                        03:38:03   Log-Likelihood:                -1901.8
No. Observations:                2858   AIC:                             3822.
Df Residuals:                    2849   BIC:                             3875.
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                 0.78

In [88]:
results = smf.ols('got_response ~ gender + asian + hispanic + aa + sector', data=response_data).fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           got_response   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     45.80
Date:                Wed, 20 Dec 2017   Prob (F-statistic):           1.74e-77
Time:                        03:30:36   Log-Likelihood:                -1900.1
No. Observations:                2858   AIC:                             3820.
Df Residuals:                    2848   BIC:                             3880.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                 0.77